In [1]:
# ─── Cell 1: Configuration ────────────────────────────────────────────
COURSE_FILE       = "data_course_for_RSv2.xlsx"
JOBS_FILE         = "combined_all_jobs_requirements.xlsx"
SHEET_NAME        = "Sheet1"

# ─── Learner Profile ─────────────────────────────────────────────────
DESIRED_JOB       = ""
COURSES_COMPLETED = [ 
]
# ─── Embedding Params & Stopwords ────────────────────────────────────
# CHUNK_SIZE        = 256
thai_stopwords    = {
}


In [2]:
# ─── Imports & Helper Functions ─────────────────────────────────
import pandas as pd
import re
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
from nltk.tokenize import word_tokenize

def preprocess_text(text):
    tokens = word_tokenize(str(text))
    return [w for w in tokens if w not in thai_stopwords]

def normalize(text: str) -> str:
    text = text.replace("\xa0", " ")
    return re.sub(r"\s+", " ", text).strip()


c:\Users\ninen\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# ─── Load & Preprocess Data ───────────────────────────────────────
courses = pd.read_excel(COURSE_FILE, sheet_name=SHEET_NAME)
jobs    = pd.read_excel(JOBS_FILE)
jobs    = jobs.rename(columns={"Job Title":"JOB_NAME", "Job Description":"REQUIREMENTS"})

courses["PROCESSED_DESC"] = courses["C_Description"].apply(preprocess_text)
jobs   ["PROCESSED_REQ"]  = jobs  ["REQUIREMENTS"].apply(preprocess_text)


In [4]:
# ─── Cell 4: Load SBERT Model ───────────────────────────────────────
from sentence_transformers import SentenceTransformer

# pick your SBERT model
MODEL_NAME = 'paraphrase-multilingual-MiniLM-L12-v2'

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load SentenceTransformer
sbert_model = SentenceTransformer(MODEL_NAME, device=device)

# enforce the 512-token max
sbert_model.tokenizer.model_max_length = 512


In [5]:
# ─── Cell 5: Batch-encode with SBERT ─────────────────────────────────
# prepare clean text
jobs   ['TEXT_CLEAN']   = jobs   ['PROCESSED_REQ'].apply(lambda toks: ' '.join(toks))
courses['TEXT_CLEAN']   = courses['PROCESSED_DESC'].apply(lambda toks: ' '.join(toks))

# encode in one go
jobs_embs    = sbert_model.encode(
    jobs['TEXT_CLEAN'].tolist(),
    batch_size=32,
    convert_to_numpy=True,
    truncation=True,
    max_length=512,
    show_progress_bar=True
)
courses_embs = sbert_model.encode(
    courses['TEXT_CLEAN'].tolist(),
    batch_size=32,
    convert_to_numpy=True,
    truncation=True,
    max_length=512,
    show_progress_bar=True
)

# attach embeddings
jobs   ['EMBEDDING']   = list(jobs_embs)
courses['EMBEDDING']   = list(courses_embs)



Batches: 100%|██████████| 5/5 [00:02<00:00,  1.86it/s]


In [6]:
# ─── Cell 6a (after you’ve computed EMBEDDING columns) ─────────────────
from sklearn.preprocessing import normalize as sk_normalize

# stack into a 2D array, normalize row-wise, then unpack back to lists
jobs_embs = np.stack(jobs["EMBEDDING"].to_list())          # shape (n_jobs, dim)
courses_embs = np.stack(courses["EMBEDDING"].to_list())    # shape (n_courses, dim)

jobs_embs    = sk_normalize(jobs_embs,    axis=1)          # each row → unit length
courses_embs = sk_normalize(courses_embs, axis=1)

jobs["EMBEDDING"]    = list(jobs_embs)                     # back into DataFrame
courses["EMBEDDING"] = list(courses_embs)


In [17]:
learner_profile = {
    "desired_job": "นักโภชนาการ", 
    "courses_completed": [
        "จริยธรรมเพื่อการดำเนินชีวิต",
        "สังคมไทยและโลกาภิวัตน์",
        "พฤติกรรมสุขภาพ",
        "การอ่านและการเขียนภาษาอังกฤษทั่วไป",
        "การบริหารงานสาธารณสุข",
        "ชีววิทยาเบื้องต้น",
        "การพัฒนาทักษะการคิด",
        "การสร้างเสริมสุขภาพ",
        "การวิจัยทางสาธารณสุข",
        "สังคมวิทยาและมานุษยวิทยาสาธารณสุข",
    ]
}

In [18]:
# ─── Cell 6b: Filter, dedupe & recommend ───────────────────────────────
from sklearn.metrics.pairwise import cosine_similarity

# 1) Exclude courses the learner has already taken
candidates = courses[~courses["C_Name"].isin(learner_profile['courses_completed'])].copy()

# 2) Compute similarity to the desired-job vector
desired_vec = jobs.loc[jobs["JOB_NAME"] == (learner_profile['desired_job']), "EMBEDDING"].iloc[0]
candidates["SIMILARITY"] = candidates["EMBEDDING"].apply(
    lambda v: cosine_similarity([desired_vec], [v])[0][0]
)

candidates_unique = candidates.drop_duplicates(subset="C_Name", keep="first")
candidates_unique = candidates_unique.sort_values(['SIMILARITY'], ascending=False).head(10)

print("Your desired job is:")
print(learner_profile['desired_job'])

print("\nRecommended Courses:")
print(candidates_unique[["C_Name", "C_Description", "SIMILARITY"]])


Your desired job is:
นักโภชนาการ

Recommended Courses:
                                         C_Name  \
54                          โภชนศาสตร์สาธารณสุข   
23                   เทคโนโลยีการเพาะเลี้ยงกุ้ง   
83   การวางแผนและการประเมินโครงการด้านสาธารณสุข   
67                       ปัญหาพิเศษทางสาธารณสุข   
126                  กฎหมายเกี่ยวกับสิ่งแวดล้อม   
84                         ชีวสถิติทางสาธารณสุข   
87                            อนามัยสิ่งแวดล้อม   
56                    อาชีวอนามัยและความปลอดภัย   
53                  สุขศึกษาและการสื่อสารสุขภาพ   
20                      ระบบสนับสนุนการตัดสินใจ   

                                         C_Description  SIMILARITY  
54   ความหมายและความสำคัญของโภชนาการต่อสุขภาพ อาหาร...    0.586454  
23   การจัดจำแนกชีววิทยาและนิเวศวิทยาของกุ้ง  การใช...    0.512087  
83   การวางแผน รูปแบบ หลักการในการวางแผน การเขียนโค...    0.492988  
67   การศึกษาปัญหาด้านสาธารณสุขโดยใช้กระบวนการวิจัย...    0.476227  
126       แนวความคิด   วิธีการต่างๆ กฎ